## Imports


In [1]:
import numpy as np
from icecream import ic
from gxgp import  Node
import operator
import inspect

In [2]:
def numpy_wrapper(func):
    """
    Crea un wrapper per una funzione di NumPy, trasformandola in una funzione Python pura.
    """
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    
    # Copia i metadati della funzione originale (se disponibili) al wrapper
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    return wrapper

In [ ]:
# ic(np.add)
# wrapper_add = numpy_wrapper(operatoK
# ic(operator.add)

tree = Node(np.multiply, [Node(np.add, [Node(10), Node('x')]), Node(2)])

multiply(add(10, x), 2)
